In [3]:
#load "Paket.fsx"
Paket.Package ["AleaTK"; "NUnit"]

<null>

In [4]:
#load "packages/Alea/Alea.fsx"
#r "packages/Alea/lib/net45/Alea.Parallel.dll"
#r "packages/AleaTK/lib/net40/AleaTK.dll"
#r "packages/NUnit/lib/net45/nunit.framework.dll"

In [5]:
open Alea
open AleaTK

let ctx = AleaTK.Context.GpuContext(0)
let batchs = 5
let batchSize = 100000UL
let error = 1e-2
let seed = 0UL

In [6]:
let points = ctx.Device.Allocate<double2>(Shape.Create(int64 batchSize))
let pi = ctx.Device.Allocate<float>(Shape.Scalar)

In [2]:
let f = <@ fun (point:double2) -> if (point.x * point.x + point.y * point.y) < 1.0 then 4.0 else 0.0 @>
let pis = Library.Map(points, (fun points -> __eval(f) points))

input.fsx(1,23): error FS0039: The type 'double2' is not defined

In [7]:
ctx.Assign(points, Library.RandomUniform<double2>())

System.Threading.Tasks.Task

In [8]:
ctx.Assign(pi, Library.ReduceMean(pis))  // has some bug

System.AggregateException: One or more errors occurred. ---> System.AggregateException: One or more errors occurred. ---> System.Exception: Value was either too large or too small for a UInt16.
Source location stack:
-> at FSI_0011+pis@3.[Double Invoke(Alea.double2)]
-> at AleaTK.ExprImpl.Map1Expr`2+<>c__DisplayClass10_0[Alea.double2,System.Double].[Double <GenerateRValue>b__0(Int64)]
-> at AleaTK.ExprImpl.ReduceExpr`1+<>c__DisplayClass13_0[System.Double].[Double <Execute>b__0(Int32)]
-> at Alea.Parallel.Agent.AgentReduce.[Void ConsumeTile[Double](Alea.Constant`1[System.Boolean], Double ByRef, Int32, System.Func`2[System.Int32,System.Double], System.Func`3[System.Double,System.Double,System.Double], Int32)]
-> at Alea.Parallel.Agent.AgentReduce.[Double ConsumeRange[Double](Alea.deviceptr`1[System.Byte], Int32, Int32, System.Func`2[System.Int32,System.Double], System.Func`3[System.Double,System.Double,System.Double])]
-> at Alea.Parallel.Agent.AgentReduce.[Double ConsumeRange[Double](Al